![교촌](../images/교촌매장찾기_F12.png)

- search() 함수

![교촌](../images/교촌매장찾기_search.png)

In [1]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import datetime
from itertools import count
import re
import xml.etree.ElementTree as ET

In [2]:
def get_request_url(url, enc='utf-8'):
    
    req = urllib.request.Request(url)

    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print ("[%s] Url Request Success" % datetime.datetime.now())
            
            try:
                rcv = response.read()
                ret = rcv.decode(enc)
            except UnicodeDecodeError:
                ret = rcv.decode(enc, 'replace')    
            
            return ret
            
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [3]:
def getKyochonAddress(sido, result):
    
    for sido2 in count():
        Kyochon_URL = 'http://www.kyochon.com/shop/domestic.asp?txtsearch=&sido1=%s&sido2=%s' % (str(sido), str(sido2+1))

        rcv_data = get_request_url(Kyochon_URL)
        if (rcv_data == None):
             return
        soupData = BeautifulSoup(rcv_data, 'html.parser')
    
        ul_tag= soupData.find('ul', attrs={'class': 'list'})
        for store_data in ul_tag.findAll('a', href=True):
            store_name = store_data.find('strong').get_text()
            store_address = store_data.find('em').get_text().strip().split('\r')[0]
            store_sido_gu = store_address.split()[:2]
            result.append([store_name] + store_sido_gu + [store_address])

In [4]:
def main():
    
    result = []

    sido_list = ['서울특별시','부산광역시','대구광역시','제주특별자치도','광주광역시',
            '울산광역시','인천광역시','세종특별자치시','경기도','강원도','경상북도',
            '경상남도','충청북도','충청남도','전라북도','전라남도','대전광역시']
    

    sido_list1 = ['서울', '경기', '인천', '강원', '충북', '충남', '경북',
            '경남', '대전', '대구', '전북', '전남', '광주', '울산', '부산',
            '제주', '세종']

    sido_alias = """서울시:서울특별시 서울:서울특별시 강원:강원도 경기:경기도 충남:충청남도  
                    충북:충청북도 경남:경상남도 경북:경상북도 전남:전라남도 전북:전라북도 
                    제주도:제주특별자치도 제주:제주특별자치도 제주시:제주특별자치도
                    세종시:세종특별자치시 세종:세종특별자치시
                    대전시:대전광역시 대전:대전광역시 대구시:대구광역시 대구:대구광역시
                    인천시:인천광역시 인천:인천광역시 광주시:광주광역시 광주:광주광역시
                    울산시:울산광역시 울산:울산광역시 부산시:부산광역시 부산:부산광역시"""
    
    sido_dict = dict(aliasset.split(':') for aliasset in sido_alias.split())

    gungu_alias = """고양시일산서구:고양시 고양시덕양구:고양시 고양시일산동구:고양시
                    부천시오정구:부천시 부천시소사구:부천시 부천시원미구:부천시
                    안산시단원구:안산시 안산시상록구:안산시
                    안양시동안구:안양시 안양시만안구:안양시
                    성남시중원구:성남시 성남시중원구:성남시 성남시분당구:성남시 성남시수정구:성남시 
                    양편군:양평군 여주군:여주시
                    수원시권선구:수원시 수원시장안구:수원시 수원시권선구:수원시 수원시영통구:수원시 수원시팔달구:수원시
                    용인시기흥구:용인시 용인시수지구:용인시 용인시처인구:용인시
                    포항시북구:포항시 포항시남구:포항시
                    창원시마산회원구:창원시 마산회원구:마산시 창원시진해구:창원시 진해시:창원시
                    창원시마산합포구:창원시 창원시회원구:창원시 창원시성산구:창원시 창원시의창구:창원시
                    상주시낙양동:상주시 상주시사벌면:상주시
                    청주시흥덕구:청주시 청주시서원구:청주시 청주시상당구:청주시 청주시청원구:청주시
                    천안시서북구:천안시 천안시동남구:천안시 
                    전주시덕진구:전주시 전주시완산구:전주시
                    성주읍:성주군 의성읍:의성군 강화읍:강화군 웅진군:옹진군
                    구좌읍:제주시 북제주군:제주시 신광로:제주시 용문로:제주시 천수로:제주시
                    군포시금정동79-1:군포시 군포시금정동79-1:군포시
                    세종특별자치시:세종시 조치원읍:세종시 
                    한솔동:세종시 도담동:세종시 도움3로:세종시 도움8로:세종시
                    나리로:세종시 갈매로:세종시 마음로:세종시 보듬3로:세종시 
                    소담1로:세종시 호려울로:세종시 누리로:세종시 달빛로:세종시
                    연기면:세종시 연동면:세종시 전의면:세종시
                    금남면:세종시 부강면:세종시 연서면:세종시 장군면:세종시
                    전동면:세종시
                    """
    gungu_dict= dict(aliasset.split(':') for aliasset in gungu_alias.split())

    print('KYOCHON ADDRESS CRAWLING START')
    i = 0
    for sido in sido_list:
        i = i + 1
        getKyochonAddress((i), result)
    kyochon_table = pd.DataFrame(result, columns=('store', 'sido', 'gungu', 'store_address'))
    kyochon_table.sido = kyochon_table.sido.apply(lambda v: sido_dict.get(v, v))
    kyochon_table.gungu = kyochon_table.gungu.apply(lambda v: gungu_dict.get(v, v))
    kyochon_table = kyochon_table.reset_index().drop_duplicates(subset='store_address', keep='first').set_index('index')
    print(kyochon_table)
    kyochon_table.to_csv("kyochon.csv",mode='w')
    del result[:]
    print('KYOCHON ADDRESS CRAWLING END')

In [5]:
main()

KYOCHON ADDRESS CRAWLING START
[2022-10-23 00:53:29.772794] Url Request Success
[2022-10-23 00:53:30.906561] Url Request Success
[2022-10-23 00:53:31.604953] Url Request Success
[2022-10-23 00:53:32.208414] Url Request Success
[2022-10-23 00:53:32.819835] Url Request Success
[2022-10-23 00:53:33.473043] Url Request Success
[2022-10-23 00:53:34.068582] Url Request Success
[2022-10-23 00:53:34.683830] Url Request Success
[2022-10-23 00:53:35.172551] Url Request Success
[2022-10-23 00:53:36.373202] Url Request Success
[2022-10-23 00:53:36.897556] Url Request Success
[2022-10-23 00:53:37.491517] Url Request Success
[2022-10-23 00:53:38.108208] Url Request Success
[2022-10-23 00:53:38.757791] Url Request Success
[2022-10-23 00:53:39.307019] Url Request Success
[2022-10-23 00:53:40.010377] Url Request Success
[2022-10-23 00:53:41.027954] Url Request Success
[2022-10-23 00:53:41.651105] Url Request Success
[2022-10-23 00:53:42.349861] Url Request Success
[2022-10-23 00:53:43.029338] Url Reque

[2022-10-23 00:55:07.609013] Url Request Success
[2022-10-23 00:55:07.997021] Url Request Success
[2022-10-23 00:55:08.575581] Url Request Success
[2022-10-23 00:55:09.023851] Url Request Success
[2022-10-23 00:55:10.376403] Url Request Success
[2022-10-23 00:55:10.897661] Url Request Success
[2022-10-23 00:55:11.906715] Url Request Success
[2022-10-23 00:55:12.298756] Url Request Success
[2022-10-23 00:55:13.051998] Url Request Success
[2022-10-23 00:55:13.993810] Url Request Success
[2022-10-23 00:55:15.470653] Url Request Success
[2022-10-23 00:55:15.905443] Url Request Success
[2022-10-23 00:55:16.321435] Url Request Success
HTTP Error 500: Internal Server Error
[2022-10-23 00:55:17.161092] Error for URL : http://www.kyochon.com/shop/domestic.asp?txtsearch=&sido1=10&sido2=19
[2022-10-23 00:55:17.195491] Url Request Success
[2022-10-23 00:55:17.535602] Url Request Success
[2022-10-23 00:55:17.901756] Url Request Success
[2022-10-23 00:55:18.343395] Url Request Success
[2022-10-23 00